In [1]:
import numpy as np
import pandas as pd
import glob 
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import tensorflow as tf

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
metadata = pd.read_csv('metadata.csv')
metadata.head()

,title,date,for,against,id
0,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",d20191112
1,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']",d20191029
2,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",d20191022
3,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",d20190917
4,Unresolved: Shifting Power in the Middle East,2019-09-12,"['Michael Doran', 'Reuel Marc Gerecht', 'Berna...","['Brett McGurk', 'Barbara Slavin']",d20190912


In [3]:
cleaned_files = glob.glob('cleaned_transcript_data/*.csv')

In [ ]:
def determine_side(name, sidedict):
    for side in sidedict:
        if name in sidedict[side] or name.upper() in sidedict[side]:
            return side
    return 'unknown'

In [4]:
def for_against_scripts(filepath):
    fid = filepath[filepath.find('20') - 1: filepath.find('csv') - 1]
    metadata.loc[metadata['id'] == fid]
    sidedict = {}
    for side in ['for', 'against']:
        sidedict[side] = metadata.loc[metadata['id'] == fid][side].item().upper()
    cleaned = pd.read_csv(filepath).drop(columns = 'Unnamed: 0')
    cleaned['side'] = cleaned['Speaker'].apply(lambda x: determine_side(x, sidedict))
    if 'unknown' in cleaned['side'].values:
        print(filepath)
        return pd.DataFrame(data = {'side': [], 'script': []})
    new_sides = []
    new_scripts = []
    last_side = 0
    for i in range(len(cleaned)):
        side = cleaned['side'][i]
        script = cleaned['Script'][i]
        if side == last_side:
            new_scripts[-1] = new_scripts[-1] + ' ' + script
        else:
            new_sides.append(side)
            new_scripts.append(script)
            last_side = side
    return pd.DataFrame(data = {'side': new_sides, 'script': new_scripts})

In [ ]:
for filepath in cleaned_files:
    df = for_against_scripts(filepath)
    fid = filepath[filepath.find('20') - 1: filepath.find('csv') - 1]
    df.to_csv('For Against Scripts/for_against_scripts_' + fid + '.csv', index = False)